In [ ]:
import os
import sys

sys.path.append(os.path.join(os.path.dirname(sys.path[0])))

from environs import Env
import pymongo
import requests
from tqdm import tqdm
import time
from itertools import product

In [ ]:
SEASONS = range(14,16) # current season f18
PLAYLIST = ['ranked-doubles', 'ranked-duels', 'ranked-standard']
COUNT = 200
TIERS = range(0, 22) #(highest tier = 21)
PRO = 'true'

RATE_LIMIT_PER_HOUR = 490

env = Env()
env.read_env('.env')
TOKEN = os.environ["TEST_TOKEN"] # Ballchasing token
HEADERS = {"Authorization": TOKEN}
URL = "https://ballchasing.com/api/replays"

In [ ]:
cluster = pymongo.MongoClient("mongodb://localhost:27017/")
database = cluster.get_database("rl_replays")

In [ ]:
ranks = [
    "bronze-1",
    "bronze-2",
    "bronze-3",
    "silver-1",
    "silver-2",
    "silver-3",
    "gold-1",
    "gold-2",
    "gold-3",
    "platinum-1",
    "platinum-2",
    "platinum-3",
    "diamond-1",
    "diamond-2",
    "diamond-3",
    "champion-1",
    "champion-2",
    "champion-3",
    "grand-champion-1",
    "grand-champion-2",
    "grand-champion-3",
    "supersonic-legend"
]

In [ ]:
SEASONS = [f'f{x}' for x in SEASONS if x != 0]

In [ ]:
file = 'iteration.txt'
if os.path.exists(file):
    with open(file, 'r') as f:
        iteration = int(f.read())
else:
    iteration = 0

In [ ]:
def print_time(start_time, pct):
    if pct == 0 or pct == 1:
        return
    
    elapsed_time = time.time() - start_time
    remaining_time = elapsed_time / pct * (1 - pct)
    
    elapsed_time_str = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
    remaining_time_str = time.strftime("%H:%M:%S", time.gmtime(remaining_time))
    print(f"Elapsed time: {elapsed_time_str}, Remaining time: {remaining_time_str} ({pct:.2%})")


In [ ]:
delay = 3600 / RATE_LIMIT_PER_HOUR
prod = product(SEASONS, PLAYLIST, TIERS)
start = time.time()
for i, (season, playlist, tier) in enumerate(prod):
    if i < iteration:
        continue
    
    rank = ranks[tier]
    print_time(start, i / (len(SEASONS) * len(PLAYLIST) * len(TIERS)))
    print(f"Starting {playlist} {season} {rank}")

    filters = {"count": COUNT, "playlist": PLAYLIST, "min-rank": rank, "max-rank": rank, 'season': season, 'pro': PRO}

    response = requests.get(URL, headers=HEADERS, params=filters)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        break

    replays = response.json()["list"]
    data = tqdm(replays, desc=f"{rank}[{tier}]")
    for replay in data:
        replay_url = replay["link"]

        response = requests.get(replay_url, headers=HEADERS)
        if response.status_code != 200: continue

        replay_data = response.json()
        replay_id = replay_data["id"]

        if database.replays.find_one({"id": replay_id}): continue

        database.replays.insert_one(replay_data)
        time.sleep(delay)

    print(f"Finished {rank}[{tier}] rank")
    
    with open(file, 'w') as f:
        f.write(str(i))

    time.sleep(delay)


os.remove(file)
print("Finished")